Artificial Neural Netwrok

Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

'2.8.2'

Importing the dataset

In [ ]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

Applying label encoder for gender column

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [ ]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


Applying OneHotEncoding to geography column

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


Splitting the set into training and testing set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


Feature scaling (Feature scaling is a must for deep learning models)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Building an ANN

Initialize ANN

In [ ]:
ann = tf.keras.models.Sequential()

Adding the input layer and first hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

Adding the input layer and second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

Adding the output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

Training the ANN

Compile the ANN

In [ ]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Training the ANN on training set

In [ ]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.7363 - accuracy: 0.5481
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4982 - accuracy: 0.7912
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4533 - accuracy: 0.7990
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4339 - accuracy: 0.8070
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4193 - accuracy: 0.8171
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4054 - accuracy: 0.8271
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3918 - accuracy: 0.8353
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3787 - accuracy: 0.8439
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3691 - accuracy: 0.8491
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3620 - accura

Making the predictions and evaluating the models

Predict the result on a single dataset

Use our ANN model to predict if the customer with the following information will avail the services of bank or not: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000


In [ ]:
ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]]))

array([[0.04921991]], dtype=float32)

In [ ]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]]))>0.5)

[[False]]


**Solution**

Therefore, our ANN model predicts that this customer avails the services of the bank

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:**  "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And these these values are to be created in the first three columns, because the dummy variables are always created in the first columns.

Predicting the test set results

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


Making the confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1580   15]
 [ 313   92]]


0.836